In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

Using TensorFlow backend.


In [3]:
import cv2
import numpy as np
import argparse
import zipfile
from tqdm import tqdm
import os
import xmltodict

In [4]:
#read video file frame by frame, beginning and ending with a timestamp
def read_video_segments(video_path,start_ms,end_ms,i):
    vid = cv2.VideoCapture(video_path)
    fps = vid.get(cv2.CAP_PROP_FPS)
    start_frame=int((start_ms/1000.0)*fps)
    end_frame=int((end_ms/1000.0)*fps)
    vid_length=0
    with tqdm(total=end_frame-start_frame+1) as pbar: #init the progressbar,with max length of the given segment
        vid.set(cv2.CAP_PROP_POS_FRAMES,start_frame)
        while(vid.isOpened()):
            ret, frame = vid.read() # if ret is false, frame has no content
            if (vid_length+start_frame)==end_frame:
                pbar.update(1)
                break
            path='../test_frames/'
            name=str(i)+'__'+str(start_frame+vid_length)+'.png'
            cv2.imwrite(path+name,frame)
            pbar.update(1) #update the progressbar
            vid_length+=1 #increase the vid_length counter

In [5]:
def get_shots(azp):
    zip_ref = zipfile.ZipFile(azp)
    inxmlstr = zip_ref.read('content.xml')
    doc = xmltodict.parse(inxmlstr)
    shots = []
    for a in doc['package']['annotations']['annotation']:
        if a['@type'] == '#Shot':
            begin_ms = int(a['millisecond-fragment']['@begin'])
            end_ms = int(a['millisecond-fragment']['@end'])

            shots.append((begin_ms, end_ms))
    return sorted(shots)

In [6]:
# with open('test.txt','w') as f:
#     print('asf',file=f)

In [8]:
videos_path='videos/'
features_path='features_videos/'

for video in os.listdir(features_path):
    if not os.path.isdir(features_path+video):
        os.mkdir(features_path+video)
    print(video)

David_Ed_Where_Do_We_Go_From_Here
Her_bluray_Szene 11_25fps
Maltsev_Sem_Occupy_Wall_Street


In [17]:
from pathlib import Path

In [28]:
app_path=str(Path('shotdetect'))
appname = os.path.basename(app_path)

In [31]:
if not os.path.exists(app_path):
    raise Exception(_("The <b>%s</b> application does not seem to be installed. Please check that it is present and that its path is correctly specified in preferences." ) % appname)

TypeError: 'str' object is not callable

In [30]:
appname

'shotdetect'

In [ ]:
    def async_process_file(self, filename, end_callback):
        appname = os.path.basename(self.app_path)
        if not os.path.exists(self.app_path):
            raise Exception(_("The <b>%s</b> application does not seem to be installed. Please check that it is present and that its path is correctly specified in preferences." ) % appname)
        if not os.path.exists(filename):
            raise Exception(_("The file %s does not seem to exist.") % filename)

        self.app_setup(filename, end_callback)

        argv = [ self.app_path ] + self.get_process_args(filename)

        if config.data.os == 'win32':
            import win32process
            kw = { 'creationflags': win32process.CREATE_NO_WINDOW }
        else:
            kw = { 'preexec_fn': subprocess_setup }

        try:
            self.process = subprocess.Popen( argv,
                                             bufsize=0,
                                             shell=False,
                                             stdout=subprocess.PIPE,
                                             stderr=subprocess.PIPE,
                                             **kw )
        except OSError as e:
            self.cleanup()
            msg = str(e.args)
            raise Exception(_("Could not run %(appname)s: %(msg)s") % locals())

        self.progress(.01, _("Processing %s") % GObject.filename_display_name(filename))

        def execute_process():
            self.convert(self.iterator())
            self.progress(.95, _("Cleaning up..."))
            self.cleanup()
            self.progress(1.0)
            end_callback()
            return True

        # Note: the "proper" way would be to use Gobject.io_add_watch,
        # but last time I tried, this had cross-platform issues. The
        # threading approach seems to work across platforms, so "if it
        # ain't broke, don't fix it".
        t=threading.Thread(target=execute_process)
        t.start()
        return self.package

In [32]:
import xml.etree.ElementTree as ET
def read_shotdetect_xml(path):
    tree = ET.parse(path)
    root=tree.getroot().findall('content')
    info={}
    timestamps=[]
    for child in root[0].iter():
    #     if child.tag == 'media':
    #         for grandchild in child.iter():
    #             if grandchild.tag == 'fps':
    #                 info['fps']=float(grandchild.text)
    #             if grandchild.tag == 'height':
    #                 info['resolution']=[int(grandchild.text)]
    #             if grandchild.tag == 'width':
    #                 info['resolution'].append(int(grandchild.text))
        if child.tag == 'shot':
            items=child.items()
            print(items)
            timestamps.append((int(items[4][1]),int(items[4][1])+int(items[2][1])-1))
            break
    return timestamps#,info

In [33]:
read_shotdetect_xml('tests/shot_detect4/result.xml')

[('id', '0'), ('fduration', '7'), ('msduration', '291'), ('fbegin', '0'), ('msbegin', '0')]


[(0, 290)]